In [ ]:
#let’s merge this network: http://doi.org/10.18119/N91597
#with this other one: http://www.ndexbio.org/#/network/1c69beff-1229-11e6-a039-06603eb7f303
#or http://www.ndexbio.org/#/network/89975ebb-6294-11e7-a03e-0ac135e8bacf

#Merge them to make a new network in which nodes that have the same name are combined.

#If there is node 1 in network 1 that has the same name as node 2 in network 2, 
#the new network should have just one node of that name that has all the attributes from node 1 AND node 2.

#As you do the merge, each node should be given an additional attribute “fromNetwork” so we can tell which network it originally came from. Values can be “network1”, “network2” or “both”

#Save to ndex

#Use Cytoscape to give it a graphic style in which the nodes are colored based on “fromNetwork”

In [1]:
import ndex2.client as nc
import io
import json
from time import sleep

from ndex2.nice_cx_network import NiceCXNetwork
import ndex2

import networkx as nx

import pandas as pd 
import numpy as np

#create a NDEx client object to access my accouont on the NDEx server
my_account="Jeffreyc"
my_password="Frenchfri3"

my_ndex=nc.Ndex2("http://public.ndexbio.org", my_account, my_password)
my_ndex.update_status()


In [2]:
#get NiceCX objects in order to add attribute
#network1 is http://doi.org/10.18119/N91597
#network2 is chosen between 1c69beff-1229-11e6-a039-06603eb7f303 and 89975ebb-6294-11e7-a03e-0ac135e8bacf
print ("chose merge network: http://doi.org/10.18119/N91597 with 1 or 2:")
x = input()
x = int(x) 

uuid_1 = '98ba6a19-586e-11e7-8f50-0ac135e8bacf'

if (x == 1):
    uuid_2 = '1c69beff-1229-11e6-a039-06603eb7f303'
    print ("merge network with http://www.ndexbio.org/#/network/1c69beff-1229-11e6-a039-06603eb7f303")
else:
    if (x == 2):
        uuid_2 = '89975ebb-6294-11e7-a03e-0ac135e8bacf'
        print ("merge network with http://www.ndexbio.org/#/network/89975ebb-6294-11e7-a03e-0ac135e8bacf")
    else:
        uuid_2 = '1c69beff-1229-11e6-a039-06603eb7f303'
        print ("default is 1. merge network with http://www.ndexbio.org/#/network/1c69beff-1229-11e6-a039-06603eb7f303")

NiceCX_network_1 = ndex2.create_nice_cx_from_server(server='http://public.ndexbio.org',  uuid = uuid_1)
NiceCX_network_2 = ndex2.create_nice_cx_from_server(server='http://public.ndexbio.org',  uuid = uuid_2)

NiceCX_network_1.print_summary()
print ("\n")
NiceCX_network_2.print_summary()

chose merge network: http://doi.org/10.18119/N91597 with 1 or 2:
1
merge network with http://www.ndexbio.org/#/network/1c69beff-1229-11e6-a039-06603eb7f303
Name: BioPlex 2.0 ( ~ 56,000 interactions )
Nodes: 10961
Edges: 56553
Node Attributes: 10961
Edge Attributes: 169659



Name: ChEMBL - High Affinity Compounds vs human targets (Commercially available)
Nodes: 3633
Edges: 11564
Node Attributes: 13275
Edge Attributes: 53075



In [3]:
#add attribute "fromNetwork" to each node in NiceCX_network_1
for node_id, node in NiceCX_network_1.get_nodes() :
    NiceCX_network_1.add_node_attribute(node_id, name='fromNetwork', values='network1')
        
#add attribute "fromNetwork" to each node in NiceCX_network_2
for node_id, node in NiceCX_network_2.get_nodes() :
    NiceCX_network_2.add_node_attribute(node_id, name='fromNetwork', values='network2')

network_1_node_number = len(list(NiceCX_network_1.nodes))
network_2_node_number = len(list(NiceCX_network_2.nodes))

print("number of nodes in network1: ", network_1_node_number)
print("number of nodes in network2: ", network_2_node_number)

number of nodes in network1:  10961
number of nodes in network2:  3633


In [4]:
#convert NiceCX network to NetworkX object 
NetworkX_network_1 = NiceCX_network_1.to_networkx()
NetworkX_network_2 = NiceCX_network_2.to_networkx()

In [6]:
#get the first node data in network1, in order to get the list of attributes
# list(NetworkX_network_1.nodes(data=True)) -- get all the node data along with the nodes; output is an array
# list(NetworkX_network_1.nodes(data=True))[0] -- get the first node data; output is an array 
# list(NetworkX_network_1.nodes(data=True))[0][1] -- get the node attributes with value; out put is a dictionary
# list(NetworkX_network_1.nodes(data=True))[0][0] -- get the node name

# create an array to have all the network_1 node name
network_1_name_list = []

for i in range (network_1_node_number):
    network_1_name_list.append(list(NetworkX_network_1.nodes(data=True))[i][0] )
    
#showing the name list is completed
print(network_1_node_number) 

10961


In [7]:
# create an array to have all the network_2 node names
network_2_name_list = []

for j in range(network_2_node_number):
    network_2_name_list.append(list(NetworkX_network_2.nodes(data=True))[j][0] )
    
# showing the complete name list
print(network_2_node_number)   

3633


In [8]:
# compare two arrays and find the same node names, create a list to keep the set
same_node_name_list = set(network_1_name_list) & set(network_2_name_list)  

number_of_same_node =len(same_node_name_list)
print ("number of same node names in the two networks: ", number_of_same_node, "\n")

for k in range (number_of_same_node):
    print(list(same_node_name_list)[k])

number of same node names in the two networks:  8 

CAMK2B
CAMK2A
GRPR
EDNRB
PAK1
EDNRA
SRC
ADAM17


In [9]:
#for the nodes from both networks, change the value of attribute "fromNetwork" to "both"

# a loop goes though all the duplicate nodes
for k in range (number_of_same_node):
    this_node_name = list(same_node_name_list)[k]    

    for id, node in NiceCX_network_2.get_nodes():
        # get this node's name
        node_name = node.get('n') 
        
        if node_name == this_node_name:    
            NiceCX_network_2.remove_node_attribute(id, 'fromNetwork')
            NiceCX_network_2.add_node_attribute(id, name='fromNetwork', values='both')

            print (node_name, " ",NiceCX_network_2.get_node_attribute(id, 'fromNetwork'))

# convert the updated NiceCX network to NetworkX
NetworkX_network_2 = NiceCX_network_2.to_networkx()

CAMK2B   {'po': 1367, 'n': 'fromNetwork', 'v': 'both'}
CAMK2A   {'po': 996, 'n': 'fromNetwork', 'v': 'both'}
GRPR   {'po': 85, 'n': 'fromNetwork', 'v': 'both'}
EDNRB   {'po': 950, 'n': 'fromNetwork', 'v': 'both'}
PAK1   {'po': 2102, 'n': 'fromNetwork', 'v': 'both'}
EDNRA   {'po': 68, 'n': 'fromNetwork', 'v': 'both'}
SRC   {'po': 507, 'n': 'fromNetwork', 'v': 'both'}
ADAM17   {'po': 1017, 'n': 'fromNetwork', 'v': 'both'}


In [10]:
# create a new NetworkX object in order to combine network1 and network2
NetworkX_combined = nx.Graph()

#to add nodes along with node attributes, should contain (data=True)
#attributes of the node in the combined network are depending on the order of join
NetworkX_combined.add_nodes_from(NetworkX_network_1.nodes(data=True))
NetworkX_combined.add_nodes_from(NetworkX_network_2.nodes(data=True))
NetworkX_combined.add_edges_from(NetworkX_network_1.edges(data=True))
NetworkX_combined.add_edges_from(NetworkX_network_2.edges(data=True))

print ("done")

done


In [11]:
#convert NetworkX object to NiceCX object
NiceCX_combined = ndex2.create_nice_cx_from_networkx(NetworkX_combined)
NiceCX_combined.print_summary()

Name: Untitled
Nodes: 14586
Edges: 61055
Node Attributes: 53408
Edge Attributes: 191114



In [12]:
#upload the combined network to my account
uri_upload_message = NiceCX_combined.upload_to("http://public.ndexbio.org", my_account, my_password)
print(uri_upload_message)

Generating CX
http://public.ndexbio.org/v2/network/d6a91639-90b0-11e9-bcaf-0ac135e8bacf


In [13]:
#update the network profile
uuid = uri_upload_message.rpartition('/')[-1]

new_profile={"name":"combined 2 networks using NetworkX", "description":"Merge 2 networks to make a new network in which nodes that have the same name are combined.", "version":"4.0"}
my_ndex.update_network_profile(uuid, new_profile)    
my_ndex.make_network_public(uuid)


''